In [18]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import pandas as pd
import numpy as np
import psycopg2
import datetime
import time
import math
import calendar

# 输入开始时间和结束时间
begin_time = '2019-08-01 00:00:00'
end_time= '2020-05-25 20:00:00'
#data = pd.read_excel('4.26气象数据合并rainfall_pre.xlsx',sheet_name='鲁岗涌')
# data = data[['data_time','mn','总磷','氨氮','水温','溶解氧','高锰酸盐指数']]
# data[['总磷','氨氮','水温','溶解氧','高锰酸盐指数']] = data[['总磷','氨氮','水温','溶解氧','高锰酸盐指数']].astype(float)
# data['mn'] = data['mn'].astype(str)
# data['mn']=data['mn'].map(lambda x:'0'+str(x) if len(str(x))==13 else x)
def query(conn, sql):
    connect = conn
    cur = connect.cursor()
    cur.execute(sql)
    index = cur.description
    result = []
    for res in cur.fetchall():
        row = {}
        for i in range(len(index)):
            row[index[i][0]] = res[i]
        result.append(row)
        connect.close()
    return result

conn = psycopg2.connect(database="create_dw",
                        user="ds_usr",
                        password="QNWZWIPN6**F", 
                        host="47.106.73.123",
                        port='5432')
SQL = "SELECT data_time,site_name,nh3n_avg,tp_avg,do_avg,codcr_avg,codmn_avg,rainfall FROM ds_wdp.wdp_rpt_site_mon_data_list where site_name in ('鲁岗涌','芦苞涌古云桥断面','芦苞涌入西南涌断面','科勒大道断面','凤岗断面','广三高速断面') and data_time between '"+begin_time+"' and '"+end_time + "'"
rows = query(conn = conn,sql = SQL)
data= pd.DataFrame(rows)
data[['nh3n_avg','tp_avg','do_avg','codcr_avg','codmn_avg','rainfall']]=data[['nh3n_avg','tp_avg','do_avg','codcr_avg','codmn_avg','rainfall']].astype(float)
data['data_time'] = pd.to_datetime(data['data_time'])
data = data.sort_values(by='data_time')
data.head()

,data_time,site_name,nh3n_avg,tp_avg,do_avg,codcr_avg,codmn_avg,rainfall
367,2019-08-01 00:00:00,鲁岗涌,0.62,0.224,8.68,NaN,4.1,0.0
749,2019-08-01 04:00:00,鲁岗涌,0.52,0.173,4.90,NaN,1.7,0.0
750,2019-08-01 08:00:00,鲁岗涌,0.47,0.191,8.77,NaN,2.1,0.0
653,2019-08-01 12:00:00,鲁岗涌,0.50,0.202,5.52,NaN,1.8,2.8
463,2019-08-01 16:00:00,鲁岗涌,0.81,0.183,8.88,NaN,1.5,0.0


In [19]:
data['next_rain'] = data['rainfall'][1:].tolist()+[0.0]

In [20]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.models import Model
from keras import layers
from keras import Input
from keras.losses import m_l
#from keras.losses import m_mse
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from keras import layers
from keras.callbacks import LearningRateScheduler
from math import pow,floor
def create_dataset(dataset,look_back,bc):
    dataX,dataY=[],[]
    for i in range(len(dataset)-look_back-bc+1):
        x = dataset[i:i+look_back,]#全部特征参与建模
        dataX.append(x)
        y = dataset[i+look_back:i+look_back+bc,0]#只抽取第一个氨氮
        dataY.append(y)
    return np.array(dataX),np.array(dataY)

In [21]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras import layers
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from tqdm import tqdm
from sklearn.externals import joblib
import math

for site in tqdm(data['site_name'].unique()):
    #选择站点
    site_data = data[data['site_name']==site]
    site_data = site_data.set_index('data_time').resample('4H').asfreq().fillna(method='ffill').fillna(method='bfill').reset_index()
    if site in ['芦苞涌古云桥断面','芦苞涌入西南涌断面','科勒大道断面','凤岗断面','广三高速断面']:
        zb_list = ['nh3n_avg','tp_avg','do_avg','codcr_avg']
    else:
        zb_list = ['nh3n_avg','tp_avg','do_avg','codmn_avg']
    
    for zb in zb_list:
        train_data = site_data[site_data['data_time'] <'2020-05-15 00:00:00'][[zb,'next_rain']]
        test_data = site_data[site_data['data_time'] >='2020-05-15 00:00:00'][[zb,'next_rain']]
        train_data = np.array(train_data)
        test_data = np.array(test_data)
        #数据标准化     
        an_train_data = train_data[:,0].reshape(-1,1)
        rain_train_data = train_data[:,1:].reshape(-1,1)
        an_test_data = test_data[:,0].reshape(-1,1)
        rain_test_data = test_data[:,1:].reshape(-1,1)
        # 仅对污染指标标准化，降雨量不进行标准化
        scaler_an = MinMaxScaler()
        scaler_an.fit(an_train_data)
        an_train = scaler_an.transform(an_train_data).reshape(-1,1)
        an_test = scaler_an.transform(an_test_data).reshape(-1,1)
        joblib.dump(scaler_an, 'lstm_model/scaler_%s_%s'%(site,zb))
        
        train_data = np.hstack([an_train,rain_train_data])
        test_data = np.hstack([an_test,rain_test_data])
        x_train,y_train = create_dataset(train_data,18,6)
        x_test,y_test = create_dataset(test_data,18,6)
        
        model = Sequential()
        model.add(layers.LSTM(32, return_sequences = True,
                             input_shape=(None, x_train.shape[2])))
        #model.add(layers.Dropout(0.2))
        model.add(layers.LSTM(64,activation='relu'))
        model.add(layers.Dense(6))
        model.compile(optimizer='adam', loss='mae')
        history = model.fit(x_train,y_train,batch_size=1024,epochs=200,verbose=0)
        model.save('lstm_model/lstm_%s_%s.h5'%(site,zb))
        y_test_predict=model.predict(x_test)
        mae = mean_absolute_error(y_test,y_test_predict)
        mse = mean_squared_error(y_test,y_test_predict)
        rmse=np.sqrt(mse)
        r2=r2_score(y_test,y_test_predict)
        print(mae,mse,rmse,r2)

  0%|          | 0/6 [00:00<?, ?it/s]

0.1259594207309013 0.028689145254333467 0.1693787036623361 -0.824145285258426
0.06479257933059208 0.010258575860725443 0.10128462795866627 -0.6973506115634933
0.13537656354222383 0.035598891781153774 0.18867668584420752 -0.10832768207497023


 17%|█▋        | 1/6 [01:48<09:04, 108.93s/it]

0.06402127781611897 0.010790391751956114 0.10387681046295229 -0.20970035715331367
0.201053672663882 0.07616089080856377 0.2759726269189823 -1.8314545328758234
0.12928528622302027 0.03913038337076853 0.1978140120688333 -3.5528247817941376
0.1099387488061323 0.022747227629404107 0.15082184069094273 -0.7671424948995397


 33%|███▎      | 2/6 [02:58<06:29, 97.26s/it] 

...

0.19883241289846632 0.12316445598558141 0.35094793913853006 -3.2941706173674254
0.05509150186841197 0.005097963340749071 0.0714000233945975 -1.3071868244871654
0.2742898200851743 0.11001787330977804 0.3316894229694068 -5.583526185177072
0.07924975514143136 0.010279864765510793 0.10138966794260051 -0.018076847402483776


 83%|████████▎ | 5/6 [06:13<01:15, 75.72s/it]

0.15618019572067304 0.037799263306688605 0.19442032637224074 -0.5934569361997969
0.05653004891312157 0.006725862889994008 0.08201135829867719 -1.4584929388625714
0.019294288605959293 0.0006755201444234278 0.025990770369949172 -0.027648837013531786
0.09103806933755654 0.013952162191246925 0.11811927104095642 -1.0451021965711746


100%|██████████| 6/6 [07:19<00:00, 73.22s/it]

0.046636629579090545 0.0038620752538476833 0.06214559722013848 -0.9326614409200525


In [22]:
from keras.models import load_model

for site in tqdm(data['site_name'].unique()):
    site_data = data[data['site_name']==site]
    site_data = site_data.set_index('data_time').resample('4H').asfreq().fillna(method='ffill').fillna(method='bfill').reset_index()
    if site in ['芦苞涌古云桥断面','芦苞涌入西南涌断面','科勒大道断面','凤岗断面','广三高速断面']:
        zb_list = ['nh3n_avg','tp_avg','do_avg','codcr_avg']
    else:
        zb_list = ['nh3n_avg','tp_avg','do_avg','codmn_avg']
    
    for zb in zb_list:
        model2 = load_model('lstm_model/lstm_%s_%s.h5'%(site,zb))
        scale_model = joblib.load('lstm_model/scaler_%s_%s'%(site,zb))
        test_data = site_data[site_data['data_time'] >='2020-05-15 00:00:00'][[zb,'next_rain']]
        test_data = np.array(test_data)
        #数据标准化     
        an_test_data = test_data[:,0].reshape(-1,1)
        rain_test_data = test_data[:,1:].reshape(-1,1)
        # 仅对污染指标标准化，降雨量不进行标准化
        an_test = scale_model.transform(an_test_data).reshape(-1,1)
        test_data = np.hstack([an_test,rain_test_data])
        x_test,y_test = create_dataset(test_data,18,6)
        
        y_test_predict=model.predict(x_test)
        y_test = scale_model.inverse_transform(y_test)
        y_test_predict = scale_model.inverse_transform(y_test_predict)
        mae = mean_absolute_error(y_test,y_test_predict)
        mse = mean_squared_error(y_test,y_test_predict)
        rmse=np.sqrt(mse)
        r2=r2_score(y_test,y_test_predict)
        print(y_test[:12])
        print(y_test_predict[:12])
        print(mae,mse,rmse,r2)

  0%|          | 0/6 [00:00<?, ?it/s]

[[0.23 0.22 0.38 0.32 0.48 0.4 ]
 [0.22 0.38 0.32 0.48 0.4  0.46]
 [0.38 0.32 0.48 0.4  0.46 0.3 ]
 [0.32 0.48 0.4  0.46 0.3  1.3 ]
 [0.48 0.4  0.46 0.3  1.3  1.3 ]
 [0.4  0.46 0.3  1.3  1.3  0.56]
 [0.46 0.3  1.3  1.3  0.56 0.53]
 [0.3  1.3  1.3  0.56 0.53 0.51]
 [1.3  1.3  0.56 0.53 0.51 0.44]
 [1.3  0.56 0.53 0.51 0.44 0.53]
 [0.56 0.53 0.51 0.44 0.53 0.43]
 [0.53 0.51 0.44 0.53 0.43 0.4 ]]
[[0.29154333 0.29178306 0.29813507 0.2924146  0.3136551  0.30264154]
 [0.28392243 0.28367904 0.290713   0.28265128 0.30353194 0.29361552]
 [0.27256742 0.27194184 0.27933124 0.26897538 0.2888726  0.28119197]
 [0.27585098 0.27452606 0.27943754 0.27191785 0.2871858  0.2832715 ]
 [0.2787749  0.27635863 0.27882162 0.27382383 0.28568053 0.2843009 ]
 [0.29797947 0.29368308 0.29182324 0.2932241  0.30003002 0.30058363]
 [0.31639272 0.31113657 0.30665162 0.31201258 0.31520355 0.31698212]
 [0.3398996  0.333408   0.3268229  0.33616897 0.33598828 0.33862424]
 [0.34845516 0.342055   0.33672193 0.34475723 0.343

 17%|█▋        | 1/6 [00:17<01:28, 17.76s/it]

[[ 5.1  4.1  5.   4.9  5.   5.8]
 [ 4.1  5.   4.9  5.   5.8  5. ]
 [ 5.   4.9  5.   5.8  5.   4.3]
 [ 4.9  5.   5.8  5.   4.3 10.8]
 [ 5.   5.8  5.   4.3 10.8 10.8]
 [ 5.8  5.   4.3 10.8 10.8  6.4]
 [ 5.   4.3 10.8 10.8  6.4  5.3]
 [ 4.3 10.8 10.8  6.4  5.3  4.3]
 [10.8 10.8  6.4  5.3  4.3  3.8]
 [10.8  6.4  5.3  4.3  3.8  4. ]
 [ 6.4  5.3  4.3  3.8  4.   5.4]
 [ 5.3  4.3  3.8  4.   5.4  5.3]]
[[4.431441  4.473199  4.5645165 4.4881077 4.67453   4.510249 ]
 [4.557097  4.57214   4.6577754 4.6072392 4.779711  4.604512 ]
 [4.5675344 4.5611405 4.6633434 4.595787  4.7620397 4.592984 ]
 [4.615019  4.5691423 4.6761384 4.6161757 4.761313  4.604768 ]
 [4.653631  4.5772223 4.6849995 4.629233  4.749913  4.613812 ]
 [4.702156  4.6077533 4.7086897 4.6631975 4.754954  4.644949 ]
 [4.840355  4.730849  4.8069053 4.798577  4.855069  4.765041 ]
 [4.906284  4.794711  4.857089  4.860042  4.896792  4.8226714]
 [4.8601522 4.7513394 4.8194423 4.8034105 4.8332176 4.7786875]
 [5.499605  5.3284054 5.290717  5.47

 33%|███▎      | 2/6 [00:35<01:10, 17.66s/it]

...

[[19.3 19.3 19.3 22.  19.2 19.6]
 [19.3 19.3 22.  19.2 19.6 22.9]
 [19.3 22.  19.2 19.6 22.9 19.2]
 [22.  19.2 19.6 22.9 19.2 20.3]
 [19.2 19.6 22.9 19.2 20.3 23. ]
 [19.6 22.9 19.2 20.3 23.  19.6]
 [22.9 19.2 20.3 23.  19.6 19.3]
 [19.2 20.3 23.  19.6 19.3 24.6]
 [20.3 23.  19.6 19.3 24.6 19.4]
 [23.  19.6 19.3 24.6 19.4 17.8]
 [19.6 19.3 24.6 19.4 17.8 18.9]
 [19.3 24.6 19.4 17.8 18.9 18.2]]
[[16.97413  16.66209  16.323074 16.457506 16.295086 15.55464 ]
 [17.04793  16.68152  16.340343 16.536995 16.37216  15.592364]
 [17.130657 16.717552 16.381205 16.62413  16.459993 15.647156]
 [17.210472 16.76071  16.433298 16.707785 16.545204 15.707149]
 [17.703014 17.427937 17.08498  17.306034 17.136448 16.386429]
 [17.780672 17.552624 17.206985 17.398827 17.254177 16.473574]
 [17.813915 17.60199  17.269978 17.426485 17.292463 16.505255]
 [18.282389 18.031914 17.639832 17.924286 17.749186 16.907927]
 [18.275558 18.0961   17.71988  17.914131 17.780256 16.914053]
 [18.276249 18.109701 17.750542 17.8

 83%|████████▎ | 5/6 [01:25<00:17, 17.28s/it]

[[32.5 31.1 32.6 33.4 23.1 25.7]
 [31.1 32.6 33.4 23.1 25.7 24.3]
 [32.6 33.4 23.1 25.7 24.3 13.7]
 [33.4 23.1 25.7 24.3 13.7 11.4]
 [23.1 25.7 24.3 13.7 11.4 15.7]
 [25.7 24.3 13.7 11.4 15.7 14.2]
 [24.3 13.7 11.4 15.7 14.2 22. ]
 [13.7 11.4 15.7 14.2 22.  24.7]
 [11.4 15.7 14.2 22.  24.7 18.3]
 [15.7 14.2 22.  24.7 18.3 11.4]
 [14.2 22.  24.7 18.3 11.4 13.1]
 [22.  24.7 18.3 11.4 13.1 18.6]]
[[24.87396  24.499094 24.646793 24.68016  25.288546 24.826866]
 [25.571247 25.121279 25.17132  25.448711 25.951729 25.454899]
 [26.466488 26.072155 26.069992 26.460295 26.835123 26.368837]
 [27.598982 27.305098 27.242575 27.712984 27.903559 27.527765]
 [28.806158 28.561783 28.43364  28.981546 29.03026  28.70726 ]
 [28.77707  28.525719 28.561073 28.85819  28.961355 28.67081 ]
 [28.525414 28.259142 28.43688  28.504211 28.619762 28.445265]
 [27.88226  27.57756  27.884253 27.744717 27.894737 27.815468]
 [26.015917 25.764088 26.307875 25.696201 25.983751 26.075426]
 [23.515398 23.296625 24.02937  22.9

100%|██████████| 6/6 [01:41<00:00, 16.92s/it]

[[ 6.6 10.3  9.9  9.   9.4  7. ]
 [10.3  9.9  9.   9.4  7.   5.4]
 [ 9.9  9.   9.4  7.   5.4  7.4]
 [ 9.   9.4  7.   5.4  7.4  7.5]
 [ 9.4  7.   5.4  7.4  7.5  9.8]
 [ 7.   5.4  7.4  7.5  9.8 12.8]
 [ 5.4  7.4  7.5  9.8 12.8  7.1]
 [ 7.4  7.5  9.8 12.8  7.1  7.1]
 [ 7.5  9.8 12.8  7.1  7.1  7. ]
 [ 9.8 12.8  7.1  7.1  7.  10.3]
 [12.8  7.1  7.1  7.  10.3 28.4]
 [ 7.1  7.1  7.  10.3 28.4 15. ]]
[[23.767448  23.80497   24.049166  24.055244  24.52328   23.65768  ]
 [21.547281  21.66861   22.304134  21.548061  22.253778  21.564926 ]
 [19.16315   19.163273  20.056541  18.821058  19.631544  19.187008 ]
 [16.788284  16.626863  17.61105   16.131908  16.98881   16.779356 ]
 [14.635423  14.326928  15.275698  13.7479105 14.619104  14.613721 ]
 [12.862522  12.455389  13.291556  11.831997  12.753905  12.862362 ]
 [11.230689  10.879717  11.568788  10.143335  11.168929  11.28778  ]
 [ 9.747123   9.57385   10.09275    8.679853   9.824196   9.861588 ]
 [ 8.710392   8.734425   9.045142   7.7494826  8.92